# Vector Databases

Em **NLP** (*Natural Language Processing*), frases e documentos são frequentemente representados como **vetores numéricos**. Isto ocorre, por exemplo, para prover integração com *Machine Learning* (modelos de **NLP** frequentemente produzem ou utilizam vetores em suas operações) e pesquisa eficiente por similaridade (ao buscar documentos ou frases semelhantes).

Um banco de dados vetorial (*Vector Database*) ou mecanismo de busca vetorial é um **banco de dados** que pode armazenar tais **vetores** (listas de números de comprimento fixo) juntamente com outros itens de dados, enquanto permite recuperar informação de forma **eficiente** e **escalável**.

Nesta aula, exploraremos desde a representação vetorial de textos até uma aplicação de **RAG** (*Retrieval-Augmented Generation*) com **LLM** (*Large Language Model*).

## Instalação das Libs

In [ ]:
# !pip -r requirements.txt

## Chave OpenAI

Nesta aula, iremos utilizar a API da OpenAI. Utilize a chave fornecida pelo professor.

**ATENÇÃO**: Utilize esta chave com cuidado. Não desperdice recursos, não execute células múltiplas vezes de forma desnecessária e, mais importante, **NÃO VAZE A CHAVE** publicamente nem compartilhe com terceiros!

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("Digite a chave da OpenAI:")

## Textos: Representação Vetorial

Vetores são representações matemáticas de dados em um espaço de alta dimensão. Nesse espaço, cada dimensão corresponde a uma característica dos dados, com o número de dimensões variando de algumas centenas a dezenas de milhares, dependendo da complexidade dos dados representados.

A maneira mais simples de vetorizar textos é utilizando o método **Bag of Words** (**BoW**). Nele, cada texto é representado pela ocorrência (ou frequência) de suas palavras. Ele ignora a ordem das palavras e considera apenas a presença ou ausência delas no documento.

Por exemplo, vamos supor um dicionário de uma língua com apenas três palavras:

```python
dic = ["oi", "bom", "horrível"]
```

Agora, considere as seguintes frases:

```python
frases = [
    "oi tudo bom?",
    "bom dia",
    "foi horrível",
]

Podemos fazer uma representação vetorial das frases pelos seguintes vetores:

```python
vetores = [
    [1, 1, 0], # Tem palavra "oi", tem "bom", não tem "horrivel"
    [0, 1, 0], # Não tem "oi", tem "bom", não tem "horrível"
    [0, 0, 1], # Não tem "oi", não tem "bom", tem "horrível"
]
```

Perceba que ignoramos as palavras não pertencentes ao dicionário.

Vamos representar isto em código-fonte!

In [ ]:
import plotly.graph_objects as go
import numpy as np

# Frases personalizadas
frases = [
    "oi tudo bom?",
    "bom dia",
    "foi horrível",
    "oi, tive um dia bom e horrível ao mesmo tempo"
]

# Vetores das frases, considerando o dicionário ["oi", "bom", "horrível"]
vetores = np.array([
    [1, 1, 0],
    [0, 1, 0],
    [0, 0, 1],
    [1, 1, 1],
])

E visualizar um gráfico dos vetores:

In [ ]:

# Cores diferentes para cada ponto
cores = ["#636EFA", "#EF553B", "#00CC96", "#666666"]

# Criar uma figura do Plotly
fig = go.Figure()

# Adicionar pontos ao gráfico
for vector, color, frase in zip(vetores, cores, frases):
    fig.add_trace(go.Scatter3d(
        x=[vector[0]],
        y=[vector[1]],
        z=[vector[2]],
        mode="markers",
        marker=dict(size=5, color=color),
        text=f"{frase} {vector}",
        hoverinfo="text"
    ))

# Definir título e rótulos dos eixos com limites
fig.update_layout(
    title="Vetores de Frases - Gráfico de Pontos 3D",
    scene=dict(
        xaxis=dict(title="Eixo oi", range=[0, 2]),
        yaxis=dict(title="Eixo bom", range=[0, 2]),
        zaxis=dict(title="Eixo horrível", range=[0, 2])
    ),
    showlegend=False
)

# Mostrar o gráfico
fig.show()


## Similaridade entre vetores

A similaridade do cosseno mede o cosseno do ângulo entre dois vetores. É uma medida popular em processamento de linguagem natural e recuperação de informações e pode ser calculada como:


$$ s(A,B)=\frac{A⋅B}{∥A∥∥B∥}$$

Veja uma definição em python

In [ ]:
import numpy as np

def cosine_similarity(vector1, vector2):
    cosine_sim = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
    return cosine_sim

**Dica**: Também é possível utilizar a função da biblioteca `scipy`:

```python
from scipy.spatial.distance import cosine
```

Vamos calcular a similaridade entre as frases:

In [ ]:
# Calcular a matriz de distância cosseno
num_frases = len(frases)
matriz_distancia = np.zeros((num_frases, num_frases))

for i in range(num_frases):
    for j in range(num_frases):
        matriz_distancia[i, j] = cosine_similarity(vetores[i], vetores[j])

E visualizar o mapa de calor:

In [ ]:
# Criar uma figura do Plotly
fig = go.Figure(data=go.Heatmap(
    z=matriz_distancia,
    x=frases,
    y=frases,
    colorscale="YlOrRd"
))

# Definir título e rótulos dos eixos
fig.update_layout(
    title="Matriz de Distância Cosseno",
    xaxis_title="Frases",
    yaxis_title="Frases"
)

# Mostrar o gráfico
fig.show()

**Exercício**: A frase `"bom dia"` é mais similar à frase `"oi tudo bom?"` ou à frase `"foi horrível"`?

In [ ]:
# Sua resposta aqui!

**Exercício**: Qual seria a representação vetorial da frase `"olá tenham todos um dia horrível"` considerando o mesmo dicionário do exemplo?

<a href="#" title="[0, 0, 1] pois apenas a palavra 'horrivel' está no dicionário e no texto!">Passe o mouse aqui para conferir resposta.</a>

In [ ]:
# Sua resposta aqui!

## Indo além do BoW: Word Embeddings

Embora seja simples e eficiente, o **BoW** tem várias limitações. Primeiramente, ele não captura a semântica das palavras. Por exemplo, as palavras `"cão"` e `"cachorro"` são sinônimos, mas no **BoW**, elas são tratadas como palavras completamente diferentes.

Além disso, **BoW** também não considera a **ordem das palavras**, o que pode ser problemático para entender o contexto. Por exemplo, as frases `"o gato mordeu o cachorro"` e `"o cachorro mordeu o gato"` têm significados muito diferentes, mas no **BoW**, elas podem ser representadas da mesma forma se contiverem as mesmas palavras.

Esses problemas levam à necessidade de representações de texto mais avançadas, que capturam a semântica e o contexto das palavras. É aqui que os *embeddings* entram em cena. *Embeddings* são representações vetoriais densas onde **palavras com significados semelhantes têm representações semelhantes**.

<img src="img/embedding.png">


Diferentemente do **BoW**, que representa cada palavra como um valor único em um vetor esparso, os *embeddings* mapeiam palavras para um espaço **vetorial contínuo** de relativa baixa dimensão. Isso permite que relações semânticas e contextuais sejam preservadas.

Por exemplo, em um espaço de *embeddings*, as palavras `"rei"` e `"rainha"` estarão próximas uma da outra, refletindo sua semelhança semântica, enquanto estarão longe de uma palavra como `"carro"`.

<style>
    .image-container {
        background-color: white;
        display: inline-block;
    }
</style>

<div class="image-container">
    <img src="img/emb_vectors.png">
</div>

Os modelos utilizados para *embedding* são geralmente treinados em grandes corpora de texto. Esses modelos aprendem as representações vetoriais das palavras com base em seu contexto de uso nas frases, capturando assim **nuances semânticas** e **sintáticas** que **BoW** não consegue.

**Dica**: os modelos modernos de redes neurais em **NLP** processam **tokens** ao invés de palavras. Um token é a menor unidade de texto que pode ser processada pelo modelo. Tokens podem ser palavras, caracteres, sinais de pontuação, símbolos ou partes de palavras.

Acesse o link https://platform.openai.com/tokenizer e experimente o tokenizador online da OpenAI. Digite alguns textos, com pontuações, e observe os tokens gerados.

<img src="img/openai_tokens.png">

### OpenAI Embeddings

Vamos utilizar a API da OpenAI para realizar o embedding de textos.

Veja mais em https://platform.openai.com/docs/guides/embeddings

Podemos especificar o modelo com:

In [ ]:
from langchain_openai import OpenAIEmbeddings

model = "text-embedding-ada-002"
embedding_model = OpenAIEmbeddings(model=model)

E utilizar para fazer o embedding de uma frase:

In [ ]:
vetor = embedding_model.embed_query("meu cahorro é muito fofo")

Conferindo o resultado:

In [ ]:
print(f"O vetor tem {len(vetor)} dimensões")

In [ ]:
vetor

Agora, vamos definir algumas frases:

In [ ]:
frases_novas = [
    "meu cachorro é muito fofo",
    "iphone 15 pro max é muito caro",
    "quero um pet que é amigo das crianças",
    "smartphones estão focando em IA",
    "apple encerra programa de carro autônomo",
]

E utilizar programação funcional para fazer o embedding de cada frase.

**Dica**: poderíamos ter utilizado laço `for`, utilizar programação funcional é só um detalhe aqui! Para cada frase, será mapeada a função `embedding_model.embed_query`.

In [ ]:
vetores_novos = list(map(embedding_model.embed_query, frases_novas))

Obtemos como resultado:

In [ ]:
print(f"A matriz possui {len(vetores_novos)} vetores")
print(f"Cada vetor possui {len(vetores_novos[0])} dimensões")
print(f"Ou seja, temos uma matriz {len(vetores_novos)}x{len(vetores_novos[0])}")

Vamos calcular a similaridade entre as frases pelo cálculo da similaridade cosseno entre os seus vetores:

Confira quais frases são mais similares entre si.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import plotly.express as px

similarity_matrix = cosine_similarity(vetores_novos)

fig = px.imshow(similarity_matrix, 
                x=frases_novas, 
                y=frases_novas, 
                color_continuous_scale="Viridis",
                labels={"x": "Frases", "y": "Frases", "color": "Similaridade"})
fig.update_layout(title="Heatmap de Similaridades Cosseno")
fig.show()

## Busca

Vamos supor que possuímos muitos textos representados de forma vetorial. Os textos poderiam ser parágrafos de diversos livros ou notícias. Se o usuário tiver alguma questão e desejar encontrar os vetores mais semelhantes à sua pergunta. Por exemplo:

```python
"Quais são os objetivos da área de tecnologia em 2024?"
```

Como isto poderia ser feito?

**Exercício**: Como você faria para realizar esta busca? Quais seriam os passos?

<a href="#" title="1) fazer embedding do texto da pergunta 2) calcular a distância entre o vetor do texto da pergunta e todos os vetores armazenados 3) retornar os mais semelhantes.">Passe o mouse aqui para conferir resposta.</a>

In [ ]:
# Sua resposta AQUI

**Exercício**: Comparar o vetor da pergunta com todos os vetores é eficiente de um ponto de vista computacional?

<a href="#" title="Não! Pois seria O(kn), onde k é o tamanho do vetor e n é a quantidade de vetores. Apesar de ser linear no número de vetores, seria ineficiente ao considerar um número elevado de vetores.">Passe o mouse aqui para conferir resposta.</a>

In [ ]:
# Sua resposta AQUI

**Exercício**: Utilizar um SGBD relacional poderia ajudar? Pense em como um SGBD realiza buscas e qual seria o impacto nesta situação

In [ ]:
# Sua resposta AQUI

## ChromaDB

Da mesma maneira que **SGBDs** como o **MySQL** implementam uma série de recursos para trabalhar com dados segundo o modelo relacional, também temos opções para bancos de dados vetoriais.

Nesta aula, iremos utilizar o [**Chroma DB**](https://www.trychroma.com/), mas temos outras opções comumente utilizadas:

- Pinecone (cloud)
- Milvus
- Qdrant
- PostgreSQL (um SGBD com módulo pgVector para vector DB)

<img src="img/chroma.svg">

Como exemplo, vamos utilizar o livro da Alice no País das Maravilhas (`data/alice.txt`).

Vamos dividir o livro em pedaços ou *chunks*:

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

chunk_size = 300

raw_documents = TextLoader("data/alice.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Veja um documento gerado:

In [ ]:
documents[0]

Então, criamos o banco de dados. Observe que iremos aplicar o `embedding_model`, que irá requerer chamadas à API da OpenAI.

Ainda, será criada uma pasta `alice_chroma_*`.

In [ ]:
from langchain_chroma import Chroma

db = Chroma.from_documents(
    documents, embedding_model, persist_directory=f"./alice_chroma_{model}"
)

Comente a linha acima que cria o `db` para evitar chamadas desnecessárias à API da OpenAI. Vamos passar a abrir direto do arquivo!

In [ ]:
from langchain_chroma import Chroma

db = Chroma(
    embedding_function=embedding_model, persist_directory=f"./alice_chroma_{model}"
)

Agora, podemos utilizar o `db` para recuperar textos cujos vetores sejam mais similares à determinada pergunta:

In [ ]:
pergunta = "Qual a cor das rosas da grande roseira?"

Para isto, será necessário calcular o vetor da pergunta (utilizando o mesmo modelo de *embedding* aplicado aos dados).

In [ ]:
embedding_pergunta = embedding_model.embed_query(pergunta)

print(f"O vetor da pergunta tem dimensão {len(embedding_pergunta)}.")

Então, podemos encontrar os `k` vetores mais similares:

In [ ]:
docs_resposta = db.similarity_search_by_vector(embedding_pergunta, k=3)

for doc in docs_resposta:
    # Exibir até 100 primeiros caracteres do conteúdo
    print(doc.page_content[:100])
    print("-"*30)

***Exercício**: teste com outras perguntas! Confira no arquivo `data/alice.txt` se as respostas fazem sentido.

In [ ]:
# Sua resposta AQUI!

Por enquanto apenas retornamos os textos cujos vetores são mais semelhantes à pergunta. O texto é um recorte do texto original e o usuário precisa ler os textos e procurar a resposta.

## RAG

**RAG** (**Retrieval Augmented Generation**), é uma técnica que combina **recuperação de informações** com um modelo **LLM** de geração de texto para melhorar os resultados. Este método fornece ao modelo informações que podem estar mais atualizadas do que o conjunto de dados usado no treinamento.]

Ainda, pode prover informações espefícicas de um contexto, por exemplo, quando queremos respostas envolvendo apenas os dados de um PDF empresarial privado.

A estrutura de uma solução RAG envolve três etapas principais:

1) Identificação de documentos relevantes que refletem o contexto da pergunta
2) Combinação desse contexto com um prompt que contém instruções específicas
3) geração do texto utilizando um LLM.

Nosso prompt será construído com:

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = """
Você é um assistente para tarefas de resposta a perguntas. Use as seguintes partes do contexto recuperado para responder à pergunta. Se você não sabe a resposta, basta dizer que não sabe. Use no máximo três frases e mantenha a resposta concisa.

Pergunta: {question} 

Contexto: {context} 

Resposta:
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["question", "context"]
)

Vamos especificar onde os textos de contexto são recuperados. Iremos utilizar o `db` (Chroma DB construído anteriormente).

In [ ]:
retriever = db.as_retriever()

Como **LLM**, iremos utilizar o modelo `gpt-3.5-turbo`.

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

E criar uma *RAG chain* com:

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


Então, podemos fazer nossas perguntas, que serão respondidas utilizando o texto da Alice como referência.

In [ ]:
rag_chain.invoke("Qual o título do livro? E do terceiro capítulo?")

Para verificar que de fato o contexto é importante, vamos criar um *retriever* vazio e tentar repetir a pergunta. Perceba que, como não temos os dados do livro da Alice, a API do ChatGPT não irá responder corretamente (faltou contexto).

In [ ]:
empty_retriever = Chroma(embedding_function=embedding_model).as_retriever()

rag_chain_no_retriever = (
    {"context": empty_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_no_retriever.invoke("Qual o título do livro? E do terceiro capítulo?")

**Dica:** Os prompts podem ser obtidos direto do *hub* langchain:

```python
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
```

Veja mais em https://smith.langchain.com/hub/rlm/rag-prompt e https://smith.langchain.com/hub/rlm

## Chainlit

Vamos criar um *app* para conversar com o livro da Alice utilizando o *chainlit* https://docs.chainlit.io/.

Rode a célula abaixo para criar um arquivo `app.py`:

In [ ]:
%%writefile app.py

import asyncio
import os
import chainlit as cl
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

load_dotenv(override=True)


def singleton(cls):
    instances = {}

    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]

    return get_instance


@singleton
class MyRAG:

    def __init__(self):

        def format_docs(docs):
            return "\n\n".join(doc.page_content for doc in docs)

        embedding_model_name = os.getenv("EMBEDDING_MODEL")
        embedding_model = OpenAIEmbeddings(model=embedding_model_name)

        num_vectors = int(os.getenv("NUM_VECTORS"))

        vector_db_dir = os.getenv("VECTOR_DB_DIR")

        retriever = Chroma(
            embedding_function=embedding_model,
            persist_directory=vector_db_dir,
        ).as_retriever(num_vectors=num_vectors)

        prompt_template = """
Você é um assistente para tarefas de resposta a perguntas. Use as seguintes partes do contexto recuperado para responder à pergunta. Se você não sabe a resposta, basta dizer que não sabe. Use no máximo três frases e mantenha a resposta concisa.

Pergunta: {question} 

Contexto: {context} 

Resposta:
"""

        prompt = PromptTemplate(
            template=prompt_template, input_variables=["question", "context"]
        )

        llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

        self.rag_chain = (
            {
                "context": retriever | format_docs,
                "question": RunnablePassthrough(),
            }
            | prompt
            | llm
            | StrOutputParser()
        )

    def invoke(self, pergunta):
        return self.rag_chain.invoke(pergunta)


@cl.on_chat_start
async def on_chat_start():
    await cl.Message(
        content="Oi!\nSou a assistente virtual da Alice no País das Maravilhas!\nMe faça perguntas sobre o livro!"
    ).send()


@cl.on_message
async def main(message: cl.Message):
    resp = MyRAG().invoke(message.content)
    await cl.Message(content=resp).send()


Preencha o arquivo `.env`a partir do `.env.example` (faça uma cópia).

Então, no terminal, inicialize a aplicação chainlit com:

```console
chainlit run app.py
```

Acesse, em seu navegador, a URL fornecida, provavelmente http://localhost:8000

Pronto, você criou uma aplicação RAG!

## Referências

- https://en.wikipedia.org/wiki/Vector_database
- https://platform.openai.com/docs/guides/embeddings
- https://platform.openai.com/tokenizer
- https://python.langchain.com/v0.1/docs/use_cases/question_answering/sources/
- https://semaphoreci.com/blog/word-embeddings
- https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/
- Imagens:
    - https://docs.trychroma.com/img/hrm4.svg
    - https://storage.googleapis.com/gweb-cloudblog-publish/images/image4_fUvNRO7.max-800x800.png
- Livro: http://www.ebooksbrasil.org/eLibris/alicep.html autorizado para uso didático conforme http://www.ebooksbrasil.org/
